In [2]:
setwd('/home/jmitchel/scITD/')
library(devtools)
load_all()

Loading required package: usethis

ℹ Loading scITD

Loading required package: Matrix



In [3]:
pbmc <- readRDS(file="/home/jmitchel/data/covid_data/COVID19_subsetted_seurat.rds")


In [5]:
table(pbmc@meta.data$majorType)



     B    CD4    CD8     DC  Macro   Mega   Mono    Neu     NK Plasma 
 95566 202502 302683  11771    588  12137 257722    566  54678   7604 

[1] 4.353846

In [6]:
colnames(pbmc@meta.data)

[1] "orig.ident"                                     
 [2] "nCount_RNA"                                     
 [3] "nFeature_RNA"                                   
 [4] "celltype"                                       
 [5] "majorType"                                      
 [6] "sampleID"                                       
 [7] "PatientID"                                      
 [8] "datasets"                                       
 [9] "City"                                           
[10] "Age"                                            
[11] "Sex"                                            
[12] "Sample.type"                                    
[13] "CoVID.19.severity"                              
[14] "Sample.time"                                    
[15] "Sampling.day..Days.after.symptom.onset."        
[16] "SARS.CoV.2"                                     
[17] "Single.cell.sequencing.platform"                
[18] "BCR.single.cell.sequencing"                     
[19] "TCR.single.cell.sequencing"                     
[20] "Outcome"                                        
[21] "Comorbidities"                                  
[22] "COVID.19.related.medication.and.anti.microbials"
[23] "Leukocytes..G.L."                               
[24] "Neutrophils..G.L."                              
[25] "Lymphocytes..G.L."                              
[26] "Unpublished"                                    
[27] "n_genes_by_counts"                              
[28] "total_counts"                                   
[29] "total_counts_mt"                                
[30] "pct_counts_mt"

In [7]:
param_list <- initialize_params(ctypes_use = c("B","CD4","CD8","NK","Mono","DC","Mega","Plasma"),
                                ncores = 30, rand_seed = 10)

pbmc_container <- make_new_container(seurat_obj=pbmc,
                                     params=param_list,
                                     metadata_cols=c('PatientID',
                                                     "Sex",
                                                     "majorType",
                                                     'Sample.type',
                                                     'CoVID.19.severity',
                                                     'Sample.time',
                                                     'Single.cell.sequencing.platform',
                                                     'Outcome',
                                                     'Comorbidities',
                                                     'COVID.19.related.medication.and.anti.microbials'),
                                     metadata_col_nm=c('donors',
                                                       'sex',
                                                       'ctypes',
                                                       'fresh_frozen',
                                                       'covid_severity',
                                                       'sample_time',
                                                       'sequencing_platform',
                                                       'outcome',
                                                       'comorbidities',
                                                       'medications'))

In [8]:
pbmc_container <- form_tensor(pbmc_container, donor_min_cells=20, gene_min_cells=0,
                              norm_method='trim', scale_factor=10000,
                              vargenes_method='norm_var_pvals', vargenes_thresh=.05,
                              scale_var = TRUE, var_scale_power = .5)

[1] "parsing data matrix by cell/tissue type..."
[1] "cleaning data..."
[1] "collapsing count matrices from cells to donors (aka pseudobulk operation)..."
[1] "normalizing data..."
[1] "calculating gene overdispersion factors..."
[1] "selecting highly variable genes from each cell type..."
[1] "scaling variance..."
[1] "forming tensor..."
[1] "Complete!"


In [10]:
dim(pbmc_container$scMinimal_ctype[[1]]$pseudobulk)

[1]   33 3795

In [9]:
pbmc_container <- run_tucker_ica(pbmc_container, ranks=c(7,20,7),
                                 tucker_type = 'regular', rotation_type = 'ica')

ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [1] not equal to array extent


In [ ]:
# get factor-meta data associations
pbmc_container <- get_meta_associations(pbmc_container,vars_test=c('sex','Age','pool','processing','Status','Ethnicity'),
                                        stat_use='pval')

# plot donor scores by status
pbmc_container <- plot_donor_matrix(pbmc_container, meta_vars=c('sex'),
                                    cluster_by_meta = 'sex',
                                    show_donor_ids = FALSE,
                                    add_meta_associations='pval')
pbmc_container <- plot_donor_matrix(pbmc_container, meta_vars=c('sex','processing','pool'),
                                    cluster_by_meta = 'pool',
                                    show_donor_ids = FALSE,
                                    add_meta_associations='pval')

pbmc_container$plots$donor_matrix